In [120]:
%reset -f

import os
import pandas as pd
import gzip
import struct
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from rasterio.warp import Resampling
from rasterio.crs import CRS
from shapely.geometry import mapping
from rasterio.transform import from_origin
from rasterio.mask import mask
import urllib.request
from datetime import date
from datetime import timedelta
import xarray as xr
import rioxarray
from tqdm import tqdm
import requests
import math



def read_persiann_css_online0(url, nrow, ncol, dtype=np.int16):
    """Downloads, decompresses a Persiann CCS .bin.gz file, converts it to a NumPy array,
        sets -9999 values to NaN, divides all values by 100, and reshapes to 2x2.

        Args:
            url: The URL of the Persiann CCS .bin.gz file.
            dtype: The desired data type for the NumPy array (default: np.float32).

        Returns:
            A reshaped NumPy array containing the processed data as a nrow*ncol matrix.

        Raises:
            URLError: If an error occurs while downloading the file.
            ValueError: If the decompressed data size is not compatible with nrow*ncol reshape.
    """
    # Try opening the URL and decompressing the data
    try:
        compressed_data = requests.get(url).content
        decompressed_data = gzip.decompress(compressed_data)
        # Convert to NumPy array
        data = np.frombuffer(decompressed_data, dtype=np.dtype('>h')).astype(float) 
        data = data.reshape((nrow,ncol))
        data_1 = data[:,int(ncol/2):]
        data_2 = data[:,:int(ncol/2)]
        data = np.hstack((data_1,data_2))
        data= data/100
        data[data < 0] = np.nan
        data = np.flipud(data)
        print(f"Data successfully downloaded from {url}")
        compressed_data = None
        decompressed_data = None
        data_1 = None
        data_2 = None
        del compressed_data
        del decompressed_data
        del data_1
        del data_2
        return data
    except urllib.error.URLError as e:
        raise urllib.error.URLError(f"Error reading file from {url}: {e}")


def read_persiann_css_online(url, nrow, ncol):
    """Downloads, decompresses a Persiann CCS .bin.gz file, converts it to a NumPy array,
       sets -9999 values to NaN, divides all values by 100, and reshapes to 2x2.
       With visualized progress bar while downloading each file

    Args:
        url: The URL of the Persiann CCS .bin.gz file.
        nrow: Number of rows for reshaping the data.
        ncol: Number of columns for reshaping the data.
        dtype: The desired data type for the NumPy array (default: np.float32).

    Returns:
        A reshaped NumPy array containing the processed data as a nrow*ncol matrix.

    Raises:
        URLError: If an error occurs while downloading the file.
        ValueError: If the decompressed data size is not compatible with nrow*ncol reshape.
    """

    try:
        # Open the URL, handle unknown content length
        with urllib.request.urlopen(url) as response:
            total_size = int(response.headers.get('content-length', None))  # Get total size (if available)
            progress_bar = tqdm(total=total_size, desc="Downloading data from "+url, unit='B', unit_scale=True, unit_divisor=1024) if total_size is not None else tqdm(desc="Downloading data")

            # Alternative 1: Read data in chunks using read(chunksize)
            compressed_data = b''
            chunksize = 1024*100 # update per 100 kb of downloading
            while True:
                chunk = response.read(chunksize)
                if not chunk:
                    break
                compressed_data += chunk
                if total_size is not None:  # Update progress if total size known
                    progress_bar.update(len(chunk))

            # Alternative 2: Read entire data (if content length is known or for small files)
            # if total_size is not None:
            #     compressed_data = response.read()

            progress_bar.close()  # Close progress bar after download

        # Decompress data
        decompressed_data = gzip.decompress(compressed_data)
        data = np.frombuffer(decompressed_data, dtype=np.dtype('>h')).astype(float) 
        data = data.reshape((nrow,ncol))
        data_1 = data[:,int(ncol/2):]
        data_2 = data[:,:int(ncol/2)]
        data = np.hstack((data_1,data_2))
        data= data/100
        data = np.round(data,2)
        data[data < 0] = np.nan
        data = np.flipud(data)
        return data
    
    except urllib.error.URLError as e:
        raise urllib.error.URLError(f"Error reading file from {url}: {e}")

def format_number_with_zeros(number, desired_digits):
  """Formats a number with leading zeros to reach the desired number of digits.

  Args:
      number: The integer to format.
      desired_digits: The desired number of digits in the output string.

  Returns:
      A string representation of the number with leading zeros if needed.
  """

  if not isinstance(number, int) or desired_digits <= 0:
    raise ValueError("Invalid input: number must be an integer and desired_digits must be positive.")

  # Convert the number to a string
  number_str = str(number)

  # Add leading zeros if needed
  num_leading_zeros = desired_digits - len(number_str)
  formatted_string = "0" * num_leading_zeros + number_str

  return formatted_string

def iter_url(start_year, start_month, start_day, end_year, end_month, end_day, interval, max_num_of_obs_per_slice):
    """
    Generates a set of URLs and corresponding date ranges based on the given parameters.

    Args:
        start_year: The starting year of the date range.
        start_month: The starting month of the date range.
        start_day: The starting day of the date range.
        end_year: The ending year of the date range.
        end_month: The ending month of the date range.
        end_day: The ending day of the date range.
        interval: The interval between URLs in hours (e.g., 3 for every 3 hours).
        max_num_of_obs_per_slice: The maximum number of observations per data slice.

    Returns:
        A list of dictionaries, each containing:
        - urls: A list of generated URLs for the slice.
        - date_slice: A pandas date range object representing the slice.
        - start_datetime (pd.Timestamp): The start datetime of the slice.
        - end_datetime (pd.Timestamp): The end datetime of the slice.
        The number of observations in the given time range combining all slices
    """

    # Define the start and end time points
    start_time = pd.Timestamp(year=start_year, month=start_month, day=start_day)
    end_time = pd.Timestamp(year=end_year, month=end_month, day=end_day, hour=23,minute=59,second=59)
    if start_time>end_time:
       raise ValueError("start date must be no more recent than the end date inputed")
    # Construct the time range with the specified interval and inclusive "left" boundary
    freq = str(interval) + 'h'
    time_range = pd.date_range(start=start_time, end=end_time, freq=freq, inclusive='left')
    ranges = []
    url_base = "https://persiann.eng.uci.edu/CHRSdata/PERSIANN-CCS/"

    # Iterate over time slices
    num_of_slice=math.ceil(len(time_range)/max_num_of_obs_per_slice)
    for i in range(num_of_slice):
        slice_start_index = i * max_num_of_obs_per_slice
        slice_end_index = min((i + 1) * max_num_of_obs_per_slice - 1, len(time_range) - 1)  # Handle cases where the last slice might be shorter
        slice_start = time_range[slice_start_index]
        slice_end = time_range[slice_end_index]
        date_slice = time_range[slice_start_index:slice_end_index + 1]  # Use direct slicing for DatetimeIndex
        urls = []
        # Iterate over all days within the slice
        for time_point in date_slice:
            doy = time_point.timetuple().tm_yday  # Day of the year
            year_2d = str(time_point.year)[-2:]  # Last two digits of the year
            doy_formatted = format_number_with_zeros(doy, 3)
            hh_formatted = format_number_with_zeros(time_point.hour, 2)  # Hour with leading zeros
            # Construct the URL
            url = url_base + str(interval) + "hrly/" + "rgccs" + freq + year_2d + doy_formatted + hh_formatted + '.bin.gz'
            urls.append(url)

        range_dict = {
            "urls": urls,
            "date_slice": date_slice,
            "start_datetime": slice_start,
            "end_datetime": slice_end,
        }
        ranges.append(range_dict)

    print(f"{len(time_range)} datasets from {start_time.date()} to {end_time.date()} with a frequency of {freq} will be downloaded from {url_base}")
    return ranges, len(time_range)

def num_to_ordinal(num):
  """
  Converts a number to its ordinal representation.

  Args:
      num: The number to be converted (must be an integer).

  Returns:
      str: The ordinal representation of the number (e.g., "1st", "2nd", "3rd", etc.).

  Raises:
      ValueError: If the input number is not an integer.
  """

  if not isinstance(num, int):
    raise ValueError("Input must be an integer")

  # Handle special cases for numbers ending in 11, 12, and 13
  if num % 100 in [11, 12, 13]:
    return str(num) + "th"

  # Get the last digit of the number
  last_digit = num % 10

  # Choose the appropriate suffix based on the last digit
  suffix = "st" if last_digit == 1 else ("nd" if last_digit == 2 else ("rd" if last_digit == 3 else "th"))

  return str(num) + suffix

def download_geopackage(url, filename="temp.gpkg"):
  """
  Downloads a geopackage file from a URL.

  Args:
      url: The URL of the online geopackage file.
      filename: Temporary filename to store the downloaded geopackage (optional).

  Returns:
      The filename of the downloaded geopackage.
  """
  # Download the geopackage file
  response = requests.get(url, stream=True)
  if response.status_code == 200:
    with open(filename, 'wb') as f:
      for chunk in response.iter_content(1024):
        f.write(chunk)
    print(f"Succeeded in downloading geopackage from {url}")
    del response
    return filename
  else:
    raise ValueError(f"Failed to download geopackage from {url}")

def clip_data(np_data, shape_esm):
    """
    Clips the DataArray 'data' using the geometry from 'shape_esm'.

    Args:
        np_data (xr.DataArray): The DataArray to clip.
        shape_esm (geopandas.GeoDataFrame): GeoDataFrame containing the geometry to clip with.

    Returns:
        xr.DataArray: The clipped DataArray.
    """

    lat = np.arange(60, -60, -0.04)  # 3000 rows
    lon = np.arange(-180, 180, 0.04)  # 9000 cols

    data = xr.DataArray(data=np_data, dims=["lat", "lon"], coords=[lat, lon])
    data.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    data.rio.write_crs("epsg:4326", inplace=True)

    data_esm = data.rio.clip(shape_esm.geometry.apply(mapping), shape_esm.crs, all_touched=True)
    lat = None
    lon = None
    data = None
    del lat
    del lon
    del data
    return data_esm 

def clip_array_with_geopackage(data, filename, crs=None):
  """
  Clips a NumPy array using a geopackage file.

  Args:
      data: The NumPy array to clip.
      filename: Path to the geopackage file.
      crs: Coordinate Reference System (CRS) of the data and shapefile (optional).

  Returns:
      A NumPy array representing the clipped data.
  """

  # Read the shapefile
  gdf = gpd.read_file(filename)

  # Get the first geometry (assuming only one shape is needed)
  geometry = mapping(gdf.iloc[0].geometry)

  # Open the raster dataset from the NumPy array
  with rasterio.open(None, "w", driver="GTiff", height=data.shape[0], width=data.shape[1], count=1, dtype=data.dtype) as src:
    src.transform = rasterio.Affine.identity  # Assuming unit transform for simplicity
    src.crs = crs if crs else CRS.from_epsg(4326)  # Default to EPSG:4326 (WGS84)
    src.write(data, 1)

    # Clip the raster by the geometry
    clipped, transform = rasterio.rasterize(shapes=[geometry], out_shape=data.shape, fill=0, transform=src.transform, crs=src.crs, resampling=Resampling.nearest)

  # Cleanup (optional, remove downloaded file after use)
  # import os
  # if os.path.exists(filename):
  #   os.remove(filename)

  return clipped

def read_persiann_ccs(file_path):
    ncols, nrows = 9000, 3000
    data = np.zeros((nrows, ncols), dtype=np.float32)  # Initialize data array

    with gzip.open(file_path, 'rb') as f:
        for i in range(nrows):
            for j in range(ncols):
                # Read two bytes from the file, big-endian format
                val = struct.unpack('>h', f.read(2))[0]
                # Convert to mm/3hr, handling the no-data value
                data[i, j] = np.nan if val == -9999 else val / 100.0

    return data
# return a numpy array
def convert_to_geotiff(data, geotiff_path):
    transform = from_origin(-180, 60, 0.04, 0.04)
    metadata = {
        'driver': 'GTiff',
        'height': data.shape[0],
        'width': data.shape[1],
        'count': 1,
        'dtype': 'float32',
        'crs': '+proj=latlong',
        'transform': transform
    }
    
    with rasterio.open(geotiff_path, 'w', **metadata) as dst:
        dst.write(data, 1)

def clip_raster_with_gpkg(raster_path, gpkg_path, clipped_raster_path):
    gdf = gpd.read_file(gpkg_path)
    gdf = gdf.to_crs(crs='+proj=latlong')
    
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, gdf.geometry, crop=True)
        out_meta = src.meta.copy()
        
        out_meta.update({
            'driver': 'GTiff',
            'height': out_image.shape[1],
            'width': out_image.shape[2],
            'transform': out_transform
        })
        
        with rasterio.open(clipped_raster_path, 'w', **out_meta) as dest:
            dest.write(out_image)

# def plot_clipped_data(clipped_raster_path):
#     with rasterio.open(clipped_raster_path) as src:
#         data = src.read(1)
#         plt.figure(figsize=(12, 6))
#         plt.imshow(data, cmap='viridis', origin='upper')
#         plt.colorbar(label='Precipitation (mm/3hr)')
#         plt.title('Clipped PERSIANN-CCS Precipitation')
#         plt.xlabel('Longitude')
#         plt.ylabel('Latitude')
#         plt.show()

def plot_clipped_data(clipped_raster_array):
    plt.figure(figsize=(12, 6))
    plt.imshow(clipped_raster_array, cmap='viridis', origin='upper')
    plt.colorbar(label='Precipitation (mm/3hr)')
    plt.title('Clipped PERSIANN-CCS Precipitation')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()

# # Paths
# file_path = "C:/Users/liang.yang/Downloads/rgccs3h0300100.bin.gz"
# geotiff_path = '/home/hidrologia/Downloads/rgccs3h0300100.tif'
# gpkg_path = '/home/hidrologia/Pronostico_hidro/Inputs/Manduriacu_delimitada.gpkg'
# clipped_raster_path = '/home/hidrologia/Downloads/clipped_rgccs3h0300100.tif'

# # Read and convert the data to GeoTIFF
# data = read_persiann_ccs(file_path)
# convert_to_geotiff(data, geotiff_path)

# # Clip the GeoTIFF with the geopackage shapefile
# clip_raster_with_gpkg(geotiff_path, gpkg_path, clipped_raster_path)

# # Plot the clipped data
# plot_clipped_data(clipped_raster_path)


In [121]:
nrow = 3000
ncol = 9000 # As per the readme file of the raw dataset
start_year=2023
start_month=1
start_day=18
end_year=2024
end_month=3
end_day=1
interval=3 # choose from 3 and 6
shape_file_url = "https://github.com/DPAINAMHI/esmeraldas/raw/bba2fdfc0fa2098b99d9e106f4775cc42b56de8a/Basins/esmeraldas.gpkg"
max_obs_per_slice=4
# Download the shape file of Esmeraldas from the github repo
shape_file = download_geopackage(shape_file_url)
shape_file=gpd.read_file(shape_file)

# Get the list of URLs and desired time range of data
slice_list, num_of_obs = iter_url(start_year,start_month,start_day,end_year,end_month,end_day,interval,max_obs_per_slice)
t_downloaded=0
for t, slice in enumerate(slice_list):

    print("\n***********************************************************")
    print(f"Start to process the {num_to_ordinal(t+1)} slice, {len(slice_list)-t-1} slices left\n")
    
    time_range = xr.DataArray(dims='time',data=slice['date_slice'])
    
    array_list=[] # Null container of all the clipped array
    for i, url in enumerate(slice['urls']):
        try:
            print(f"Start to download the {num_to_ordinal(i+1)} file of the {num_to_ordinal(t+1)} slice from {url}, {max_obs_per_slice-i-1} files left in this slice")
            array_esm_temp = clip_data(read_persiann_css_online0(url, nrow, ncol), shape_file)
            t_downloaded+=1
            array_list.append(array_esm_temp)
            array_esm_temp = None
            del array_esm_temp
            del url
            print(f"Succeeded in storing data of {time_range[i].data} \n{t_downloaded} of {num_of_obs} downloaded")
        except Exception as e:
            print(f"Error processing URL {url}: {e}")    
            time_range = time_range.drop_isel({"time":i}) 
    dataset = xr.concat(array_list, dim=time_range.variable)
    dataset = dataset.drop_vars('spatial_ref')
    data_dict = {"precipitation": dataset}
    ds = xr.Dataset(data_dict)
    ds = ds.astype(np.float32)
    dataset = None
    data_dict = None
    del dataset
    del data_dict
    slice_start = str(slice['start_datetime'].date())+"HH"+format_number_with_zeros(slice['start_datetime'].hour,2)
    slice_start = slice_start.replace('-','')
    slice_end = str(slice['end_datetime'].date())+"HH"+format_number_with_zeros(slice['end_datetime'].hour,2)
    slice_end = slice_end.replace('-','')
    dest_folder_name = "clipped_ds_"+str(interval)+'h'
    # Check if the new folder exists
    if not os.path.exists(dest_folder_name):
        # Create the new folder if it already exists in the current work dir
        os.makedirs(dest_folder_name)
    # Define the filename for the dataset
    output_filename = slice_start + '__' + slice_end +'.nc'
    # Combine the folder path and filename
    full_path = os.path.join(dest_folder_name, output_filename)
    ds.to_netcdf(full_path)
    ds = None
    del ds
    slice = None
    del slice
    array_list = None
    del array_list
    time_range = None
    del time_range
    print(f"Clipped arrays from {slice_start} to {slice_end} saved to {full_path}")
    print(f"This is the {num_to_ordinal(t+1)} file of {len(slice_list)} in total")

Succeeded in downloading geopackage from https://github.com/DPAINAMHI/esmeraldas/raw/bba2fdfc0fa2098b99d9e106f4775cc42b56de8a/Basins/esmeraldas.gpkg
3272 datasets from 2023-01-18 to 2024-03-01 with a frequency of 3h will be downloaded from https://persiann.eng.uci.edu/CHRSdata/PERSIANN-CCS/

***********************************************************
Start to process the 1st slice, 817 slices left

Start to download the 1st file of the 1st slice from https://persiann.eng.uci.edu/CHRSdata/PERSIANN-CCS/3hrly/rgccs3h2301800.bin.gz, 3 files left in this slice
Data successfully downloaded from https://persiann.eng.uci.edu/CHRSdata/PERSIANN-CCS/3hrly/rgccs3h2301800.bin.gz
Succeeded in storing data of 2023-01-18T00:00:00.000000000 
1 of 3272 downloaded
Start to download the 2nd file of the 1st slice from https://persiann.eng.uci.edu/CHRSdata/PERSIANN-CCS/3hrly/rgccs3h2301803.bin.gz, 2 files left in this slice
Data successfully downloaded from https://persiann.eng.uci.edu/CHRSdata/PERSIANN-CCS

KeyboardInterrupt: 

In [240]:
ds = xr.open_dataset('clipped_ds_6h/20230101HH00__20230101HH00.nc')
a=ds.precipitation[0].data


In [ ]:
ds.isel(ds.precipitation.argmax(...)) # Method for conditional selecting
ds.precipitation[0].data # Method for slicing or indexing, never use .values

In [82]:
da = xr.DataArray(time_range)
v = da.variable


<xarray.DataArray (time: 2)> Size: 16B
array(['2023-01-18T00:00:00.000000000', '2023-01-18T09:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 16B 2023-01-18 2023-01-18T09:00:00

In [107]:
da=xr.DataArray(dims='time',data=time_range.data)
da.variable

<xarray.Variable (time: 128)> Size: 1kB
array(['2023-01-18T00:00:00.000000000', '2023-01-18T03:00:00.000000000',
       '2023-01-18T06:00:00.000000000', '2023-01-18T09:00:00.000000000',
       '2023-01-18T12:00:00.000000000', '2023-01-18T15:00:00.000000000',
       '2023-01-18T18:00:00.000000000', '2023-01-18T21:00:00.000000000',
       '2023-01-19T00:00:00.000000000', '2023-01-19T03:00:00.000000000',
       '2023-01-19T06:00:00.000000000', '2023-01-19T09:00:00.000000000',
       '2023-01-19T12:00:00.000000000', '2023-01-19T15:00:00.000000000',
       '2023-01-19T18:00:00.000000000', '2023-01-19T21:00:00.000000000',
       '2023-01-20T00:00:00.000000000', '2023-01-20T03:00:00.000000000',
       '2023-01-20T06:00:00.000000000', '2023-01-20T09:00:00.000000000',
       '2023-01-20T12:00:00.000000000', '2023-01-20T15:00:00.000000000',
       '2023-01-20T18:00:00.000000000', '2023-01-20T21:00:00.000000000',
       '2023-01-21T00:00:00.000000000', '2023-01-21T03:00:00.000000000',
       '2023-01-21T06:00:00.000000000', '2023-01-21T09:00:00.000000000',
       '2023-01-21T12:00:00.000000000', '2023-01-21T15:00:00.000000000',
       '2023-01-21T18:00:00.000000000', '2023-01-21T21:00:00.000000000',
       '2023-01-22T00:00:00.000000000', '2023-01-22T03:00:00.000000000',
       '2023-01-22T06:00:00.000000000', '2023-01-22T09:00:00.000000000',
       '2023-01-22T12:00:00.000000000', '2023-01-22T15:00:00.000000000',
       '2023-01-22T18:00:00.000000000', '2023-01-22T21:00:00.000000000',
...
       '2023-01-29T06:00:00.000000000', '2023-01-29T09:00:00.000000000',
       '2023-01-29T12:00:00.000000000', '2023-01-29T15:00:00.000000000',
       '2023-01-29T18:00:00.000000000', '2023-01-29T21:00:00.000000000',
       '2023-01-30T00:00:00.000000000', '2023-01-30T03:00:00.000000000',
       '2023-01-30T06:00:00.000000000', '2023-01-30T09:00:00.000000000',
       '2023-01-30T12:00:00.000000000', '2023-01-30T15:00:00.000000000',
       '2023-01-30T18:00:00.000000000', '2023-01-30T21:00:00.000000000',
       '2023-01-31T00:00:00.000000000', '2023-01-31T03:00:00.000000000',
       '2023-01-31T06:00:00.000000000', '2023-01-31T09:00:00.000000000',
       '2023-01-31T12:00:00.000000000', '2023-01-31T15:00:00.000000000',
       '2023-01-31T18:00:00.000000000', '2023-01-31T21:00:00.000000000',
       '2023-02-01T00:00:00.000000000', '2023-02-01T03:00:00.000000000',
       '2023-02-01T06:00:00.000000000', '2023-02-01T09:00:00.000000000',
       '2023-02-01T12:00:00.000000000', '2023-02-01T15:00:00.000000000',
       '2023-02-01T18:00:00.000000000', '2023-02-01T21:00:00.000000000',
       '2023-02-02T00:00:00.000000000', '2023-02-02T03:00:00.000000000',
       '2023-02-02T06:00:00.000000000', '2023-02-02T09:00:00.000000000',
       '2023-02-02T12:00:00.000000000', '2023-02-02T15:00:00.000000000',
       '2023-02-02T18:00:00.000000000', '2023-02-02T21:00:00.000000000'],
      dtype='datetime64[ns]')

In [13]:
import xarray as xr

# Sample data
data = np.arange(12).reshape(3, 4)
da = xr.DataArray(data, dims=("x", "y"), name="data")
print(da)

# Define the index and dimension to drop along
index_to_remove = 2  # Index to remove
dim_to_drop = 'x'  # Dimension along which to drop (can be 'y' or other dimensions)

# Drop the element using drop_sel
da_new = da.drop_sel({dim_to_drop: index_to_remove})

print(da_new)

<xarray.DataArray 'data' (x: 3, y: 4)> Size: 48B
array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])
Dimensions without coordinates: x, y
<xarray.DataArray 'data' (x: 2, y: 4)> Size: 32B
array([[0, 1, 2, 3],
       [4, 5, 6, 7]])
Dimensions without coordinates: x, y


In [ ]:
lat=np.arange(60,-60,-0.04) # 3000 rows
lon=np.arange(-180,180,0.04) # 9000 cols
data = xr.DataArray(data=data, dims=["lat", "lon"], coords=[lat,lon])
data.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
data.rio.write_crs("epsg:4326", inplace=True)
data_esm = data.rio.clip(shape_esm.geometry.apply(mapping),shape_esm.crs,all_touched=True)

In [ ]:

fig = plt.subplots(figsize=(8,8))
plt.imshow(clipped,cmap='cividis',vmin=0.1,vmax=12)
plt.title("Precipitación Esmeraldas PERSIANN-CCS")
plt.colorbar(label='Precipitación (mm)')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [208]:
def read_css_local0(item):
    f = gzip.GzipFile(item) # item is the local dic of the .bin.gz file
    file_content = f.read()
    data = np.frombuffer(file_content, dtype=np.dtype('>h')).astype(float) 
    data = data.reshape((3000,9000))
    data_1 = data[:,4500:]
    data_2 = data[:,:4500]
    data = np.hstack((data_1,data_2))
    data= data/100
    data[data < 0] = np.nan
    data = np.flipud(data)
    return data

def read_persiann_css_online1(url, nrow, ncol):
    try:
        decompressed_data = gzip.decompress(requests.get(url).content)
        # Convert to NumPy array
        data = np.frombuffer(decompressed_data, dtype=np.dtype('>h')).astype(float) 
        data = data.reshape((3000,9000))
        data_1 = data[:,4500:]
        data_2 = data[:,:4500]
        data = np.hstack((data_1,data_2))
        data= data/100
        data[data < 0] = np.nan
        data = np.flipud(data)
        print(f"Data successfully downloaded from {url}")
        return data
    except urllib.error.URLError as e:
        raise urllib.error.URLError(f"Error reading file from {url}: {e}")

In [209]:
item = "C:/Users/liang.yang/Downloads/rgccs6h2300100.bin.gz"
shape_esm=gpd.read_file('C:/Users/liang.yang/Desktop/esmeraldas/Basins/esmeraldas.gpkg')
d1=read_css_local0(item)
lat=np.arange(60,-60,-0.04) # 3000 rows
lon=np.arange(-180,180,0.04) # 9000 cols
data = xr.DataArray(data=d1, dims=["lat", "lon"], coords=[lat,lon])
data.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
data.rio.write_crs("epsg:4326", inplace=True)
data_esm_1 = data.rio.clip(shape_esm.geometry.apply(mapping),shape_esm.crs,all_touched=True)

In [194]:
import gzip
import numpy as np
import urllib.request
import io

# URL of the .bin.gz file
url = 'https://persiann.eng.uci.edu/CHRSdata/PERSIANN-CCS/3hrly/rgccs3h2300100.bin.gz'

decompressed_data = gzip.decompress(requests.get(url).content)


In [196]:
# Convert to NumPy array
data = np.frombuffer(decompressed_data, dtype=np.dtype('>h')).astype(float) 
data = data.reshape((3000,9000))
data_1 = data[:,4500:]
data_2 = data[:,:4500]
data = np.hstack((data_1,data_2))
data= data/100
data[data < 0] = np.nan
data = np.flipud(data)


In [197]:

# Convert the decompressed data to a NumPy array

clipped_online=clip_data(data,shape_esm)
clipped_online=clipped_online.data